In [1]:
# pip install pyvis

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from pyvis import network as net
from pyvis.network import Network

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [3]:
def features_eng(df, version):
    """
    
    """
    if version == 'clustering':
        df = df.drop(columns=['dist2','TransactionID'])
    elif version == 'anomaly':
        df = df.drop(columns=['dist2','customer_id','TransactionID'])
    elif version == 'network':
        df = df.drop(columns=['dist2'])
    df = df.rename(columns={'id_31':'browser'})
    df['P_emaildomain'] = df['P_emaildomain'].mask(df['P_emaildomain']=='gmail','gmail.com')
    df['R_emaildomain'] = df['R_emaildomain'].mask(df['R_emaildomain']=='gmail','gmail.com')
    df['id_30'] = df['id_30'].replace(" ","_",regex=True)
    df['id_30'] = df['id_30'].str.replace(".","_",regex=False)
    df['browser'] = df['browser'].mask(df['browser'].str.contains('SM') | df['browser'].str.contains('ZTE'),'other')
    df['browser'] = df['browser'].astype(str)
    df['browser_enc'] = 'other'
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('ie'),'ie')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('safari'),'safari')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('edge'),'edge')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('firefox'),'firefox')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('android'),'android')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('Android'),'android')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('chrome'),'chrome')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('opera'),'opera')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('google'),'chrome')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('samsung'),'android')
    df['DeviceType'] = df['DeviceType'].mask(df['browser']== 'ie 11.0 for tablet','tablet')
    df = df.drop(columns='browser')
    df['device_info'] = df['DeviceInfo'].replace("-","_",regex=True)
    df['device_info2'] = df['device_info'].replace(" ","_",regex=True)
    df['device_info'] = df['DeviceInfo'].replace("-","_",regex=True)
    df['device_info2'] = df['device_info'].replace(" ","_",regex=True)
    df['device_info3'] = df['device_info2'].str.split("_").str[0]
    df['device_info3'] = df['device_info3'].str.lower()
    df['device_info_v4'] = 'other'
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='windows') | (df['device_info3']=='microsoft') | (df['device_info3']=='trident/7.0')),'windows')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='ios') | (df['device_info3']=='iphone')),'ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='blade','blade')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='lenovo','lenovo')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='redmi','redmi')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='pixel','pixel')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='android','android')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='alcatel','alcatel')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='nokia','nokia')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='asus','asus')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='oneplus','oneplus')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='zte','zte')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='hisense','hisense')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='linux','linux')
    df['device_info_v4'] = df['device_info_v4'].mask((df['device_info3'].str.contains("lg") | (df['device_info3'].str.contains("nexus"))),'lg')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3'].str.contains("huawei")) | (df['device_info3'].str.contains("hi6210sft"))),'huawei')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='motog3') |(df['device_info3']=='moto')),'moto')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='sm') |(df['device_info3']=='samsung')),'samsung')
    df = df.drop(columns=['device_info','device_info2','device_info3','DeviceInfo'])
    return df

In [4]:
df = pd.read_pickle("../../../data_lake/output_prod/df_under.pkl")

In [5]:
df.sample(2)

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist2,P_emaildomain,R_emaildomain,M4,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,num_accounts_related_to_user,num_days_previous_transaction,customer_id,num_transaction_per_time,multi_transaction_per_time
114734,3428185,0,200.0,R,6019,583.0,150.0,visa,226.0,credit,123.0,87.0,NaN,gmail.com,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Windows 10,edge 16.0,24.0,1920x1080,match_status:2,T,F,T,T,desktop,Windows,1.0,0.000,1054,1,0
16071,3036158,0,25.0,H,12570,462.0,150.0,visa,226.0,debit,272.0,87.0,NaN,gmail.com,NaN,NaN,166.0,14.0,IP_PROXY:HIDDEN,15.0,NaN,147.0,Found,Found,Found,Windows 7,chrome 62.0,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0,0.625,9376,1,0


In [6]:
df = features_eng(df,'network')

In [7]:
df.sample(3)

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,M4,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,num_accounts_related_to_user,num_days_previous_transaction,customer_id,num_transaction_per_time,multi_transaction_per_time,browser_enc,device_info_v4
119300,3453178,0,100.0,R,12839,321.0,150.0,visa,226.0,debit,264.0,87.0,yahoo.com,yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Android_7_0,24.0,2560x1440,match_status:2,T,F,T,F,mobile,1.0,93.875,2147,1,0,chrome,samsung
34975,3075236,0,150.0,H,7585,553.0,150.0,visa,226.0,credit,325.0,87.0,gmail.com,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS_11_2_1,32.0,2436x1125,match_status:1,T,F,T,T,mobile,1.0,0.000,6645,1,0,safari,ios
99544,3353806,0,100.0,R,3278,453.0,150.0,visa,226.0,debit,387.0,87.0,sbcglobal.net,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Windows_7,24.0,1440x900,match_status:2,T,F,T,T,desktop,1.0,0.000,32258,1,0,ie,windows


In [8]:
df = df[['customer_id','TransactionID','TransactionAmt','DeviceType','device_info_v4','browser_enc','isFraud']]

In [9]:
df.sample(3)

,customer_id,TransactionID,TransactionAmt,DeviceType,device_info_v4,browser_enc,isFraud
115642,32221,3433341,25.000,desktop,windows,edge,0
81324,28976,3249766,25.000,mobile,ios,safari,0
53371,72,3106502,32.182,mobile,moto,chrome,0


In [10]:
df = df.dropna(axis=0)

In [11]:
df.shape

(140811, 7)

In [12]:
df = df.sample(frac=0.001)

In [13]:
df.shape

(141, 7)

In [14]:
df['isFraud'].value_counts()

0    130
1     11
Name: isFraud, dtype: int64

In [15]:
for i in df:
    if i != 'TransactionAmt':
        df[i] = df[i].astype(str)

In [16]:
rel1 = ['customer_id','TransactionID','has done']
rel2 = ['customer_id','browser_enc','using']
rel3 = ['customer_id','device_info_v4','on']
rel4 = ['customer_id','DeviceType','with']

In [17]:
# rel1 = ['customer_id','TransactionID','has done']
# rel2 = ['TransactionID','browser_enc','using']
# rel3 = ['browser_enc','device_info_v4','on']
# rel4 = ['device_info_v4','DeviceType','with']

In [18]:
# rel1 = ['customer_id','TransactionID','has done']
# rel2 = ['TransactionID','DeviceType','using']
# rel3 = ['DeviceType','device_info_v4','on']
# rel4 = ['device_info_v4','browser_enc','with']

In [19]:
# rel1 = ['customer_id','DeviceType','using']
# rel2 = ['DeviceType','device_info_v4','on']
# rel3 = ['device_info_v4','browser_enc','with']
# rel4 = ['browser_enc','TransactionID','has done']

In [20]:
# rel1 = ['customer_id','DeviceType','using']
# rel2 = ['DeviceType','device_info_v4','on']
# rel3 = ['device_info_v4','browser_enc','with']
# rel4 = ['customer_id','TransactionID','has done'] # ok

In [21]:
rels = [rel1,rel2,rel3,rel4]

In [22]:
# rels = [rel4]

In [23]:
dt = dict()
for i in range(len(rels)):
    dt[i] = dict()
    dt[i]['source'] = rels[i][0]
    dt[i]['target'] = rels[i][1]
    dt[i]['rel'] = rels[i][2]

In [24]:
db = []

In [25]:
for i in df.index:
    # select one row 
    row = df.loc[df.index==i]
    # create empty dataframe to be filled with info taken from the row selected
    gp = pd.DataFrame(columns=['source','target','rel'])
    # for loop for taking index and name of the dict of the relationship
    for y in dt.keys():
        for z in dt[y].keys():
            # for the rel value, we need to extract the relatioship value from the dict not from the row of the dataframe! 
            if z == 'rel':
                gp.at[y,z] = dt[y][z]
            else:
                gp.at[y,z] = row[dt[y][z]][i]
    db.append(gp)

In [26]:
final = pd.concat(db)
final = final.reset_index().drop(columns='index')
final = final.drop_duplicates()
final.shape

(544, 3)

Extraction of the fraud only 

In [27]:
db2 = []

In [28]:
df2 = df[df['isFraud']=='1']

In [29]:
df2.shape

(11, 7)

In [30]:
rel5 = ['customer_id','TransactionID','has done']
rels = [rel5]
dt2 = dict()
for i in range(len(rels)):
    dt2[i] = dict()
    dt2[i]['source'] = rels[i][0]
    dt2[i]['target'] = rels[i][1]
    dt2[i]['rel'] = rels[i][2]

In [31]:
for i in df2.index:
    # select one row 
    row2 = df2.loc[df2.index==i]
    # create empty dataframe to be filled with info taken from the row selected
    gp2 = pd.DataFrame(columns=['source','target','rel'])
    # for loop for taking index and name of the dict of the relationship
    for y in dt2.keys():
        for z in dt2[y].keys():
            # for the rel value, we need to extract the relatioship value from the dict not from the row of the dataframe! 
            if z == 'rel':
                gp2.at[y,z] = dt2[y][z]
            else:
                gp2.at[y,z] = row2[dt2[y][z]][i]
    db2.append(gp2)

In [32]:
final2 = pd.concat(db2)
final2 = final2.reset_index().drop(columns='index')
final2 = final2.drop_duplicates()
final2.shape

(11, 3)

In [125]:
net = Network(directed=True,height='900px', width='100%',notebook=False,cdn_resources='in_line',select_menu=True, filter_menu=True)

In [126]:
# net.show_buttons(filter_=['physics'])

In [127]:
nodes = list(pd.unique(final[['source','target']].values.ravel('K')))

In [128]:
nodes2 = list(pd.unique(final2[['source','target']].values.ravel('K')))

In [37]:
for e in nodes:
    if e in nodes2:
        net.add_node(e,color='red')
    else:
        net.add_node(e,color='green')

In [114]:
for i in final.index:
    row = final.loc[final.index==i]
    net.add_edge(row['source'].item(),row['target'].item(),title=row['rel'].item(),label=row['rel'].item())

AssertionError: non existent node '589'

In [39]:
# net.toggle_physics(False)
net.show("kg_v99.html")

PermissionError: [Errno 1] Operation not permitted: 'bindings'

test multi partito 

In [129]:
cli = list(df['customer_id'].unique())
tra = list(df['TransactionID'].unique())
dt = list(df['DeviceType'].unique())
di = list(df['device_info_v4'].unique())
be = list(df['browser_enc'].unique())

In [53]:
# g = nx.Graph()

In [54]:
# g.add_nodes_from(cli,bipartite=0)
# g.add_nodes_from(tra,bipartite=1)
# g.add_nodes_from(dt,bipartite=2)
# g.add_nodes_from(di,bipartite=3)
# g.add_nodes_from(be,bipartite=4)

In [55]:
# [g.add_edges_from(x,y) for x,y in final]

In [56]:
# for x in final.iterrows():
#     g.add_edge(x[1].values[0],x[1].values[1])

In [57]:
# def position_MultiPartiteGraph( Graph, Parts ):
#   # Graph is a networkX Graph object, where the nodes have attribute 'agentType' with part name as a value
#   # Parts is a list of names for the parts (to be shown as columns)
#   # returns list of dictionaries with keys being networkX Nodes, values being x,y coordinates for plottingxPos = {}
#   xPos = {}
#   yPos = {}
#   for index1, agentType in enumerate(Parts):
#      xPos[agentType] = index1
#      yPos[agentType] = 0

#   pos = {}
#   for node, attrDict in Graph.nodes(data=True):
#      agentType = attrDict['bipartite']
#      # print ('node: %s\tagentType: %s' % (node, agentType))
#      # print ('\t(x,y): (%d,%d)' % (xPos[agentType], yPos[agentType]))
#      pos[node] = (xPos[agentType], yPos[agentType])
#      yPos[agentType] += 1

#   return pos

In [58]:
# nx.draw(g,pos=position_MultiPartiteGraph(g, [0,1,2,3,4]))
# plt.show()

In [59]:
# net.from_nx(g)

In [60]:
# net = Network(directed=False,height='900px', width='100%',notebook=False,cdn_resources='in_line',select_menu=True, filter_menu=True)

In [61]:
# net.show("kg_v8.html")

In [92]:
g = nx.Graph()

In [93]:
g.add_nodes_from(cli,bipartite=0)
g.add_nodes_from(tra,bipartite=1)
g.add_nodes_from(dt,bipartite=2)
g.add_nodes_from(di,bipartite=3)
g.add_nodes_from(be,bipartite=4)

In [101]:
tuples_list = [(row[1]['source'],row[1]['target']) for row in final.iterrows()]

In [102]:
# for x in final.iterrows():
#     g.add_edge(x[1].values[0],x[1].values[1])

In [103]:
g.add_edges_from(tuples_list)

In [104]:
type(g)

networkx.classes.graph.Graph

In [107]:
nt = Network()

In [108]:
nt.from_nx(g)
nt.show("nx.html")

PermissionError: [Errno 1] Operation not permitted: 'bindings'

Another test

In [130]:
for e in nodes:
    if e in cli and e in nodes2:
        net.add_node(e,color='red',bipartite=0)
    else:
        net.add_node(e,color='green',bipartite=0)

In [131]:
for e in nodes:
    if e in tra and e in nodes2:
        net.add_node(e,color='red',bipartite=0)
    else:
        net.add_node(e,color='green',bipartite=0)

In [132]:
for e in nodes:
    if e in dt and e in nodes2:
        net.add_node(e,color='red',bipartite=0)
    else:
        net.add_node(e,color='green',bipartite=0)

In [133]:
for e in nodes:
    if e in di and e in nodes2:
        net.add_node(e,color='red',bipartite=0)
    else:
        net.add_node(e,color='green',bipartite=0)

In [134]:
for e in nodes:
    if e in be and e in nodes2:
        net.add_node(e,color='red',bipartite=0)
    else:
        net.add_node(e,color='green',bipartite=0)

In [135]:
for i in final.index:
    row = final.loc[final.index==i]
    net.add_edge(row['source'].item(),row['target'].item(),title=row['rel'].item(),label=row['rel'].item())

In [136]:
net.bipartite_layout(align='vertical',dist=2,orientation='bottom_top')

AttributeError: 'Network' object has no attribute 'bipartite_layout'

In [121]:
net.show("kg_v999.html")

PermissionError: [Errno 1] Operation not permitted: 'bindings'

In [123]:
import pyvis
pyvis.__version__

'0.3.1'